In [95]:
import pandas as pd
print("Codespaces fonctionne !")

Codespaces fonctionne !


In [96]:
import os
os.chdir('/workspaces/projet-iadata')  # Change le répertoire vers la racine
print(os.getcwd())  # Vérif
#qlq soucis avec les chemins


/workspaces/projet-iadata


In [97]:
#IMPORTER LES DATASET
conso= pd.read_csv('./data/brut/consommation-quotidienne-brute.csv')

print(conso.head()) #colums
print(conso.info()) #infos de la columns
print(conso.describe())
#constat :le fichier semble mal séparé (toute la ligne est import sur une meme colonne)

  Date - Heure;Date;Heure;Consommation brute gaz (MW PCS 0°C) - GRTgaz;Statut - GRTgaz;Consommation brute gaz (MW PCS 0°C) - Teréga;Statut - Teréga;Consommation brute gaz totale (MW PCS 0°C);Consommation brute électricité (MW) - RTE;Statut - RTE;Consommation brute totale (MW)
0  2024-04-30T21:30:00+00:00;30/04/2024;23:30;;;;...                                                                                                                                                                                                                                
1  2024-04-30T21:00:00+00:00;30/04/2024;23:00;225...                                                                                                                                                                                                                                
2  2024-04-30T20:30:00+00:00;30/04/2024;22:30;;;;...                                                                                                                     

## NETTOYAGE DE DONNEES

### VERIFIER LES SEPARATEURS

In [98]:
#use sep=";" pour mieux organisé le fichier
conso=pd.read_csv('./data/brut/consommation-quotidienne-brute.csv', sep=';')

#verif des colonnes
print(conso.head())
print(conso.info())
print(conso.describe())

                Date - Heure        Date  Heure  \
0  2024-04-30T21:30:00+00:00  30/04/2024  23:30   
1  2024-04-30T21:00:00+00:00  30/04/2024  23:00   
2  2024-04-30T20:30:00+00:00  30/04/2024  22:30   
3  2024-04-30T20:00:00+00:00  30/04/2024  22:00   
4  2024-04-30T19:30:00+00:00  30/04/2024  21:30   

   Consommation brute gaz (MW PCS 0°C) - GRTgaz Statut - GRTgaz  \
0                                           NaN             NaN   
1                                       22574.0       Définitif   
2                                           NaN             NaN   
3                                       24265.0       Définitif   
4                                           NaN             NaN   

   Consommation brute gaz (MW PCS 0°C) - Teréga Statut - Teréga  \
0                                           NaN             NaN   
1                                        1562.0       Définitif   
2                                           NaN             NaN   
3                     

##### RENOMMER LES COLONNNES

In [99]:
# conso.columns['Date','Heure','Consommation', 'source','Unité']
# print(conso.head())
print(conso.columns)


Index(['Date - Heure', 'Date', 'Heure',
       'Consommation brute gaz (MW PCS 0°C) - GRTgaz', 'Statut - GRTgaz',
       'Consommation brute gaz (MW PCS 0°C) - Teréga', 'Statut - Teréga',
       'Consommation brute gaz totale (MW PCS 0°C)',
       'Consommation brute électricité (MW) - RTE', 'Statut - RTE',
       'Consommation brute totale (MW)'],
      dtype='object')


In [100]:
#certains noms sont trop long
# Renommer les colonnes clés pour simplifier leur manipulation
conso.rename(columns={
    'Date - Heure': 'DateHeure',
    'Date': 'Date',
    'Heure': 'Heure',
    'Consommation brute gaz totale (MW PCS 0°C)': 'ConsommationGazTotale',
    'Consommation brute électricité (MW) - RTE': 'ConsommationElectricite',
    'Consommation brute totale (MW)': 'ConsommationTotale',
    'Consommation brute gaz (MW PCS 0°C) - GRTgaz': 'ConsommationGazGRTgaz',
    'Consommation brute gaz (MW PCS 0°C) - Teréga': 'ConsommationGazTerega',
    'Statut - GRTgaz': 'StatutGRTgaz',
    'Statut - Teréga': 'StatutTerega',
    'Statut - RTE': 'StatutRTE'
}, inplace=True)


# Vérif des nouvelles colonnes
print(conso.columns)


Index(['DateHeure', 'Date', 'Heure', 'ConsommationGazGRTgaz', 'StatutGRTgaz',
       'ConsommationGazTerega', 'StatutTerega', 'ConsommationGazTotale',
       'ConsommationElectricite', 'StatutRTE', 'ConsommationTotale'],
      dtype='object')


In [101]:
# Calculer le pourcentage de valeurs nulles par colonne
null_percentage = conso.isnull().mean() * 100

# Afficher les colonnes avec leurs pourcentages
print(null_percentage)

DateHeure                   0.000000
Date                        0.000000
Heure                       0.000000
ConsommationGazGRTgaz      50.000000
StatutGRTgaz               50.000000
ConsommationGazTerega      50.001388
StatutTerega               50.000000
ConsommationGazTotale      50.001850
ConsommationElectricite     0.000000
StatutRTE                   0.000000
ConsommationTotale         50.001850
dtype: float64


##### Gerer les valeurs maquantes

In [102]:
# Calculer le pourcentage de valeurs nulles par colonne
null_percentage = conso.isnull().mean() * 100

# Afficher les colonnes avec leurs pourcentages
print(null_percentage)

DateHeure                   0.000000
Date                        0.000000
Heure                       0.000000
ConsommationGazGRTgaz      50.000000
StatutGRTgaz               50.000000
ConsommationGazTerega      50.001388
StatutTerega               50.000000
ConsommationGazTotale      50.001850
ConsommationElectricite     0.000000
StatutRTE                   0.000000
ConsommationTotale         50.001850
dtype: float64


In [103]:
# uniquement les lignes contenant des valeurs NaN  dles colonnes importantes
conso = conso.dropna(subset=['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale'])
print(conso.info())  # Vérif que NaN ont été supprimés


<class 'pandas.core.frame.DataFrame'>
Index: 108092 entries, 1 to 216191
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DateHeure                108092 non-null  object 
 1   Date                     108092 non-null  object 
 2   Heure                    108092 non-null  object 
 3   ConsommationGazGRTgaz    108092 non-null  float64
 4   StatutGRTgaz             108044 non-null  object 
 5   ConsommationGazTerega    108092 non-null  float64
 6   StatutTerega             107996 non-null  object 
 7   ConsommationGazTotale    108092 non-null  float64
 8   ConsommationElectricite  108092 non-null  int64  
 9   StatutRTE                108092 non-null  object 
 10  ConsommationTotale       108092 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 9.9+ MB
None


##### colonnes inutiles

In [104]:
#drop les colonnes inutiles
conso.drop(columns=['StatutGRTgaz', 'StatutTerega', 'StatutRTE'], inplace=True)
print(conso.info())

<class 'pandas.core.frame.DataFrame'>
Index: 108092 entries, 1 to 216191
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DateHeure                108092 non-null  object 
 1   Date                     108092 non-null  object 
 2   Heure                    108092 non-null  object 
 3   ConsommationGazGRTgaz    108092 non-null  float64
 4   ConsommationGazTerega    108092 non-null  float64
 5   ConsommationGazTotale    108092 non-null  float64
 6   ConsommationElectricite  108092 non-null  int64  
 7   ConsommationTotale       108092 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 7.4+ MB
None


In [105]:
#verif final

print(conso.isnull().sum())

DateHeure                  0
Date                       0
Heure                      0
ConsommationGazGRTgaz      0
ConsommationGazTerega      0
ConsommationGazTotale      0
ConsommationElectricite    0
ConsommationTotale         0
dtype: int64


##### DATA ABERRANTES

In [106]:
# Recalculer la consommation totale en sommant le gaz et l'électricité
conso['ConsommationTotale'] = conso['ConsommationGazTotale'] + conso['ConsommationElectricite']

# Vérifiez les premières lignes pour confirmer le recalcul
print(conso[['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale']])


        ConsommationGazTotale  ConsommationElectricite  ConsommationTotale
1                     24136.0                    46363             70499.0
3                     26023.0                    46046             72069.0
5                     28137.0                    46335             74472.0
7                     29044.0                    47090             76134.0
9                     28886.0                    48479             77365.0
...                       ...                      ...                 ...
216183                57082.0                    49160            106242.0
216185                56048.0                    52496            108544.0
216187                55465.0                    55531            110996.0
216189                55444.0                    56230            111674.0
216191                55353.0                    59610            114963.0

[108092 rows x 3 columns]


In [107]:
#calcul de la médiane
# Calculer la médiane pour chaque colonne concernée
median_gaz = conso['ConsommationGazTotale'].median()
median_elec = conso['ConsommationElectricite'].median()

In [108]:
# Moyenne et écart-type pour détect les aberrations
#mean : moyenne
#std : ecart-type 
mean_gaz = conso['ConsommationGazTotale'].mean()
std_gaz = conso['ConsommationGazTotale'].std() 

mean_elec = conso['ConsommationElectricite'].mean()
std_elec = conso['ConsommationElectricite'].std()


In [109]:

# Remplacement des valeurs aberrantes par la médiane
conso.loc[
    (conso['ConsommationGazTotale'] > mean_gaz + 3 * std_gaz) | 
    (conso['ConsommationGazTotale'] < mean_gaz - 3 * std_gaz),
    'ConsommationGazTotale'
] = median_gaz

conso.loc[
    (conso['ConsommationElectricite'] > mean_elec + 3 * std_elec) | 
    (conso['ConsommationElectricite'] < mean_elec - 3 * std_elec),
    'ConsommationElectricite'
] = median_elec

In [110]:
#A partir des nouvelles data
# Recalculer la consommation totale en sommant le gaz et l'électricité
conso['ConsommationTotale'] = conso['ConsommationGazTotale'] + conso['ConsommationElectricite']

# Vérifiez les premières lignes pour confirmer le recalcul
print(conso[['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale']])

        ConsommationGazTotale  ConsommationElectricite  ConsommationTotale
1                     24136.0                    46363             70499.0
3                     26023.0                    46046             72069.0
5                     28137.0                    46335             74472.0
7                     29044.0                    47090             76134.0
9                     28886.0                    48479             77365.0
...                       ...                      ...                 ...
216183                57082.0                    49160            106242.0
216185                56048.0                    52496            108544.0
216187                55465.0                    55531            110996.0
216189                55444.0                    56230            111674.0
216191                55353.0                    59610            114963.0

[108092 rows x 3 columns]


SAUVEGARDE DES DATA CLEAN DANS UN NEW FILE

In [111]:
# Vérification des modifications
print(conso.describe())

conso.to_csv('./data/prepared/conso_clean.csv', index=False) 

       ConsommationGazGRTgaz  ConsommationGazTerega  ConsommationGazTotale  \
count          108092.000000          108092.000000          108092.000000   
mean            49332.526070            3128.145626           52168.161034   
std             26435.145296            2094.177422           27985.778979   
min               719.000000             153.000000            1877.000000   
25%             26537.000000            1350.000000           27908.000000   
50%             42275.000000            2454.000000           44746.250000   
75%             68969.250000            4557.250000           73249.000000   
max            296157.000000           13621.000000          137818.000000   

       ConsommationElectricite  ConsommationTotale  
count            108092.000000       108092.000000  
mean              53396.696166       105564.857200  
std               11727.652923        38754.452701  
min               28738.000000        42928.000000  
25%               44501.000000  